<a href="https://colab.research.google.com/github/AniruddhSrivatsa/AniruddhSrivatsa/blob/master/cross_sell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df_train=pd.read_csv("train.csv")
df_train.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
0,1,Male,44.0,1.0,28.0,0.0,> 2 Years,Yes,40454.0,26.0,217.0,1.0
1,2,Male,76.0,1.0,3.0,0.0,1-2 Year,No,33536.0,26.0,183.0,0.0
2,3,Male,47.0,1.0,28.0,0.0,> 2 Years,Yes,38294.0,26.0,27.0,1.0
3,4,Male,21.0,1.0,11.0,1.0,< 1 Year,No,28619.0,152.0,203.0,0.0
4,5,Female,29.0,1.0,41.0,1.0,< 1 Year,No,27496.0,152.0,39.0,0.0


In [ ]:
df_train.shape

(37960, 12)

In [ ]:
df_test=pd.read_csv("test.csv")
df_test.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,381110,Male,25,1,11.0,1,< 1 Year,No,35786.0,152.0,53
1,381111,Male,40,1,28.0,0,1-2 Year,Yes,33762.0,7.0,111
2,381112,Male,47,1,28.0,0,1-2 Year,Yes,40050.0,124.0,199
3,381113,Male,24,1,27.0,1,< 1 Year,Yes,37356.0,152.0,187
4,381114,Male,27,1,28.0,1,< 1 Year,No,59097.0,152.0,297


In [ ]:
df_test.tail()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
127032,508142,Female,26,1,37.0,1,< 1 Year,No,30867.0,152.0,56
127033,508143,Female,38,1,28.0,0,1-2 Year,Yes,28700.0,122.0,165
127034,508144,Male,21,1,46.0,1,< 1 Year,No,29802.0,152.0,74
127035,508145,Male,71,1,28.0,1,1-2 Year,No,62875.0,26.0,265
127036,508146,Male,41,1,29.0,1,1-2 Year,No,27927.0,124.0,231


In [ ]:
df_train.tail()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage,Response
37955,37956,Male,49.0,1.0,28.0,0.0,1-2 Year,Yes,2630.0,26.0,269.0,0.0
37956,37957,Male,64.0,1.0,8.0,1.0,1-2 Year,No,41440.0,124.0,281.0,0.0
37957,37958,Female,26.0,1.0,45.0,0.0,< 1 Year,No,31444.0,152.0,221.0,0.0
37958,37959,Male,59.0,1.0,37.0,1.0,1-2 Year,No,34283.0,26.0,110.0,0.0
37959,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_test.shape

(127037, 11)

In [ ]:
print(df_train.isna().sum())
print(df_test.isna().sum())

id                      0
Gender                  1
Age                     1
Driving_License         1
Region_Code             1
Previously_Insured      1
Vehicle_Age             1
Vehicle_Damage          1
Annual_Premium          1
Policy_Sales_Channel    1
Vintage                 1
Response                1
dtype: int64
id                      0
Gender                  0
Age                     0
Driving_License         0
Region_Code             0
Previously_Insured      0
Vehicle_Age             0
Vehicle_Damage          0
Annual_Premium          0
Policy_Sales_Channel    0
Vintage                 0
dtype: int64


In [ ]:
df_train1=df_train.copy()

In [ ]:
df_train1.drop("Response",axis=1,inplace=True)

In [ ]:
df_join=pd.concat([df_train1,df_test],axis=0)
df_join.head()

,id,Gender,Age,Driving_License,Region_Code,Previously_Insured,Vehicle_Age,Vehicle_Damage,Annual_Premium,Policy_Sales_Channel,Vintage
0,1,Male,44.0,1.0,28.0,0.0,> 2 Years,Yes,40454.0,26.0,217.0
1,2,Male,76.0,1.0,3.0,0.0,1-2 Year,No,33536.0,26.0,183.0
2,3,Male,47.0,1.0,28.0,0.0,> 2 Years,Yes,38294.0,26.0,27.0
3,4,Male,21.0,1.0,11.0,1.0,< 1 Year,No,28619.0,152.0,203.0
4,5,Female,29.0,1.0,41.0,1.0,< 1 Year,No,27496.0,152.0,39.0


In [ ]:
df_join.duplicated().sum()

0

In [ ]:
df_join.shape

(164997, 11)

In [ ]:
df_join1=df_join.copy()

In [ ]:
df_join1_cat=df_join1.select_dtypes(include="O")
df_join1_cat.head()

,Gender,Vehicle_Age,Vehicle_Damage
0,Male,> 2 Years,Yes
1,Male,1-2 Year,No
2,Male,> 2 Years,Yes
3,Male,< 1 Year,No
4,Female,< 1 Year,No


In [ ]:
col_names=df_join1_cat.columns.to_list()

In [ ]:
from sklearn.preprocessing import LabelEncoder


In [ ]:
le=LabelEncoder() 

In [ ]:
for col in col_names:
 df_join1[col]=le.fit_transform(df_join1[col])

In [ ]:
df_join1.head()

In [ ]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()

In [ ]:
df_join1.drop("id",inplace=True,axis=1)

In [ ]:
df_join1.shape

In [ ]:
X_train=df_join1[0:381109]
X_train=sc.fit_transform(X_train)

In [ ]:
X_test=df_join1[381109:508146]
X_test=sc.transform(X_test)

In [ ]:
X_train[:10]

In [ ]:
import xgboost as xgb

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
xgb.XGBClassifier().get_params()

In [ ]:
xgb_params={"learning_rate":[0.1],"n_estimators":[50],"subsample":[1]}

In [ ]:
random_cv_xgb=RandomizedSearchCV(xgb.XGBClassifier(),param_distributions=xgb_params,n_iter=15,cv=7,n_jobs=-1,random_state=0)

In [ ]:
import time

In [ ]:
start=time.time()

In [ ]:
random_cv_xgb.fit(X_train,df_train["Response"])

In [ ]:
end=time.time()

In [ ]:
print(end-start)

In [ ]:
random_cv_xgb.best_estimator_

In [ ]:
random_cv_xgb.best_score_

In [ ]:
y_pred=random_cv_xgb.predict(X_test)

In [ ]:
submission4=df_test[["id"]]
poll4=pd.DataFrame(y_pred)
submission4=pd.concat([submission4,poll4],axis=1)
submission4.rename(columns={0:"Response"},inplace=True)
submission4.to_csv("kills1.csv",index=False)